In [1]:
import time
start_time = time.time()

In [2]:
# Import needed packages
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import KBinsDiscretizer
import seaborn as sns; sns.set()
import glob
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
import re
import nltk
from langdetect import detect
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
from lime import lime_text
import gensim
import gensim.downloader as gensim_api
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
import transformers
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import EnglishStemmer
import spacy
from spacy import displacy
from wordcloud import WordCloud
from sklearn.preprocessing import KBinsDiscretizer
from nbpep8.nbpep8 import pep8


# 1. Load the data

In [3]:
#Set the working directory
os.chdir('C:\\Users\\piotr\\OneDrive - Erasmus University Rotterdam\\Thesis\\')
# #Load the data
merged_df = pd.read_csv('Master Thesis Data\\merged_df.csv')

Possible speed improvements 

- drop the longest questions

- drop the columns that are not needed (answer body)

## 1.1 Create text df with Post_Body only

In [4]:
# ## Create text df with Post_Body and index only
# text_df = merged_df[['Post_Body']]

In [5]:
# text_df.to_csv('Master Thesis Data\\text_df.csv', index=True)

In [6]:
# ## Create unprocessed_df with other columns than Post_Body
# unprocessed_df = merged_df.drop(['Post_Body'], axis=1)

In [7]:
# unprocessed_df.to_csv('Master Thesis Data\\unprocessed_df.csv', index=True)

In [8]:
#Read the text_df
text_df = pd.read_csv('Master Thesis Data\\text_df.csv')

In [9]:
#Read the unprocessed_df
unprocessed_df = pd.read_csv('Master Thesis Data\\unprocessed_df.csv')

In [10]:
# #See if the data frames can be merged
# validation_df = pd.merge(text_df, unprocessed_df, on='Unnamed: 0')

## 2. Remove the html tags and other chunks of code

Remove the html tags and the content between them

In [11]:
def remove_code(x):
    """Function based on the Beautifulsoup library intended to replace 
    the content of all the <code> </code> tags of a text specified as a parameter.

    Parameters
    ----------------------------------------
    x : string
        Sequence of characters to modify.
    ----------------------------------------
    """
    soup = BeautifulSoup(x,"lxml")
    code_to_remove = soup.findAll("code")
    for code in code_to_remove:
        code.replace_with(" ")
    return str(soup)

The data set is too large to be processed at once, so it will be processed in chunks 

In [12]:
chunksize = 10 ** 4
first_one = True
filepath = os.path.join('Master Thesis Data', 'text_df.csv')  

for chunk in pd.read_csv(filepath, chunksize=chunksize):
    chunk.loc[:, 'Post_Body'] = chunk['Post_Body'].apply(remove_code)
    chunk.loc[:, 'Post_Body'] = chunk['Post_Body'].apply(lambda x: BeautifulSoup(x, "lxml").get_text())
    
    # If it's the first chunk we're processing, we want to write to a new file with the header
    if first_one:
        chunk.to_csv('Master Thesis Data\\text_df_clean.csv', mode='w', index=False)
        first_one = False
    # Otherwise, we want to append to the existing file, and we don't want to write the header again
    else:
        chunk.to_csv('Master Thesis Data\\text_df_clean.csv', mode='a', index=False, header=False)


c:\Users\piotr\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [21]:
text_df_clean.shape

(3040730, 2)

In [13]:
#Read text_df_clean
text_df_clean = pd.read_csv('Master Thesis Data\\text_df_clean.csv')

In [14]:
# #Load the data again after removing code
# merged_df = pd.read_csv('Master Thesis Data\\merged_df_clean.csv')

## 3. Remove languages other than English

Now we need to check whether the texts of the questions are written in different languages. 

In [15]:
# merged_df['Post_Body']

In [23]:
# Create feature "lang" with langdetect library
from langdetect import detect

def detect_lang(x):
    try:
        return detect(x)
    except:
        return 'unknown'  # instead of pass

In [25]:
import pandas as pd
import os
import time
from bs4 import BeautifulSoup
from langdetect import detect

chunksize = 10 ** 5
first_one = True
start_chunk = 28  # 0-indexed, adjust this to the chunk where you want to start

input_filepath = os.path.join('Master Thesis Data', 'text_df_clean.csv')  
output_filepath = os.path.join('Master Thesis Data', 'text_df_clean_3.csv')  # output to a new file

# Get the total number of rows in the file
total_rows = sum(1 for row in open(input_filepath)) - 1  # Subtract 1 to account for the header

start = time.time()

skiprows = range(1, start_chunk * chunksize + 1)  # Calculate the rows to skip based on the start_chunk

for i, chunk in enumerate(pd.read_csv(input_filepath, skiprows=skiprows, chunksize=chunksize, header=0)):
    chunk.loc[:,'short_Post_Body'] = chunk['Post_Body'].apply(lambda x: x[0:100] if isinstance(x, str) else x)
    chunk.loc[:,'lang'] = chunk['short_Post_Body'].apply(detect_lang)
    
    # If it's the first chunk we're processing, we want to write to a new file with the header
    if first_one:
        chunk.to_csv(output_filepath, mode='w', index=False)
        first_one = False
    # Otherwise, we want to append to the existing file, and we don't want to write the header again
    else:
        chunk.to_csv(output_filepath, mode='a', index=False, header=False)
    
    # Estimate remaining time
    elapsed_time = time.time() - start
    rows_processed = chunksize * (i + 1)
    print(f"Processed {rows_processed} rows in {elapsed_time:.2f} seconds.")
    
    if rows_processed < total_rows:
        remaining_rows = total_rows - rows_processed
        remaining_chunks = -(-remaining_rows // chunksize)  # Ceiling division
        estimated_time = (elapsed_time / rows_processed) * remaining_rows
        print(f"Estimated remaining time: {estimated_time:.2f} seconds or approximately {estimated_time/60:.2f} minutes.")


Processed 100000 rows in 379.83 seconds.
Estimated remaining time: 110795.50 seconds or approximately 1846.59 minutes.
Processed 200000 rows in 756.14 seconds.
Estimated remaining time: 109903.42 seconds or approximately 1831.72 minutes.
Processed 300000 rows in 880.76 seconds.
Estimated remaining time: 85050.05 seconds or approximately 1417.50 minutes.


In [26]:
#Read text_df_clean_2
text_df_clean_2 = pd.read_csv('Master Thesis Data\\text_df_clean_2.csv')
text_df_clean_2

,Unnamed: 0,Post_Body,short_Post_Body,lang
0,0,I'm taking a course on Ruby on Rails. In the v...,I'm taking a course on Ruby on Rails. In the v...,en
1,1,I have an ERP system which generates statement...,I have an ERP system which generates statement...,en
2,2,Given this example from plotly documentation\n...,Given this example from plotly documentation\n...,en
3,3,I have a folder structure like this:\n\nI have...,I have a folder structure like this:\n\nI have...,en
4,4,In my Android Studio project I have a fragment...,In my Android Studio project I have a fragment...,en
...,...,...,...,...
2799995,2799995,I need to check the resistance of a test point...,I need to check the resistance of a test point...,en
2799996,2799996,I'm trying to make a bridge from Modbus til Ho...,I'm trying to make a bridge from Modbus til Ho...,en
2799997,2799997,I am referring the book Transformer and Induct...,I am referring the book Transformer and Induct...,en
2799998,2799998,I'm trying to set up an eNB with srsRAN for ou...,I'm trying to set up an eNB with srsRAN for ou...,en


In [27]:
#Read text_df_clean_3
text_df_clean_3 = pd.read_csv('Master Thesis Data\\text_df_clean_3.csv')
text_df_clean_3

,Unnamed: 0,Post_Body,short_Post_Body,lang
0,2800000,Do they even exist?\nI looking for a solution ...,Do they even exist?\nI looking for a solution ...,en
1,2800001,Electronics newbie here and looking for a bit ...,Electronics newbie here and looking for a bit ...,en
2,2800002,I'm working on a design that requires that I t...,I'm working on a design that requires that I t...,en
3,2800003,If you have a sequential circuit in verilog or...,If you have a sequential circuit in verilog or...,en
4,2800004,I want to use the PT2262 remote control encode...,I want to use the PT2262 remote control encode...,en
...,...,...,...,...
240725,3040725,"When reading the Opcodes for the EVM article, ...","When reading the Opcodes for the EVM article, ...",en
240726,3040726,am building Dapp based on microservice and am ...,am building Dapp based on microservice and am ...,en
240727,3040727,"so have a contract deployed already, and ETH i...","so have a contract deployed already, and ETH i...",en
240728,3040728,I have two contracts that implement . One is ...,I have two contracts that implement . One is ...,en


In [28]:
#Concatenate text_df_clean_2 and text_df_clean_3
text_df_clean_2 = pd.concat([text_df_clean_2, text_df_clean_3], axis=0)
text_df_clean_2

,Unnamed: 0,Post_Body,short_Post_Body,lang
0,0,I'm taking a course on Ruby on Rails. In the v...,I'm taking a course on Ruby on Rails. In the v...,en
1,1,I have an ERP system which generates statement...,I have an ERP system which generates statement...,en
2,2,Given this example from plotly documentation\n...,Given this example from plotly documentation\n...,en
3,3,I have a folder structure like this:\n\nI have...,I have a folder structure like this:\n\nI have...,en
4,4,In my Android Studio project I have a fragment...,In my Android Studio project I have a fragment...,en
...,...,...,...,...
240725,3040725,"When reading the Opcodes for the EVM article, ...","When reading the Opcodes for the EVM article, ...",en
240726,3040726,am building Dapp based on microservice and am ...,am building Dapp based on microservice and am ...,en
240727,3040727,"so have a contract deployed already, and ETH i...","so have a contract deployed already, and ETH i...",en
240728,3040728,I have two contracts that implement . One is ...,I have two contracts that implement . One is ...,en


In [29]:
pd.DataFrame(text_df_clean_2.lang.value_counts())

,lang
en,2906364
ru,70188
es,40649
pt,7084
fr,1545
ca,1407
da,997
no,976
it,974
bg,661


The majority of the posts are in English, so we are gonna remove posts in other languages. We lose only 134366 body texts

In [31]:
#Number of posts in language other than English
print(3040730 - 2906364)

134366


In [32]:
# Deletion of data that is not in the English language
text_df_clean_2 = text_df_clean_2[text_df_clean_2['lang']=='en']

In [33]:
text_df_clean_2.shape

(2906364, 4)

In [34]:
#Drop the short_Post_Body and lang columns
text_df_clean_2 = text_df_clean_2.drop(['short_Post_Body', 'lang'], axis=1)

In [35]:
#Write down text_df_clean_2
text_df_clean_2.to_csv('Master Thesis Data\\text_df_clean_2.csv', index=False)

In [36]:
'''
Preprocess a string.
:parameter
    :param text: string - name of column containing text
    :param lst_stopwords: list - list of stopwords to remove
    :param flg_stemm: bool - whether stemming is to be applied
    :param flg_lemm: bool - whether lemmitisation is to be applied
:return
    cleaned text
'''
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and   
    #characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [37]:
lst_stopwords = nltk.corpus.stopwords.words("english")

In [38]:
import pandas as pd
import os
import time

chunksize = 10 ** 5
first_one = True

input_filepath = os.path.join('Master Thesis Data', 'text_df_clean_2.csv')  
output_filepath = os.path.join('Master Thesis Data', 'processed_df.csv')  

# Get the total number of rows in the file
total_rows = sum(1 for row in open(input_filepath)) - 1  # Subtract 1 to account for the header

start = time.time()

for i, chunk in enumerate(pd.read_csv(input_filepath, chunksize=chunksize)):
    chunk['Post_Body_clean'] = chunk['Post_Body'].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
    
    # If it's the first chunk we're processing, we want to write to a new file with the header
    if first_one:
        chunk.to_csv(output_filepath, mode='w', index=False)
        first_one = False
    # Otherwise, we want to append to the existing file, and we don't want to write the header again
    else:
        chunk.to_csv(output_filepath, mode='a', index=False, header=False)
    
    # Estimate remaining time
    elapsed_time = time.time() - start
    rows_processed = chunksize * (i + 1)
    print(f"Processed {rows_processed} rows in {elapsed_time:.2f} seconds.")
    
    if rows_processed < total_rows:
        remaining_rows = total_rows - rows_processed
        remaining_chunks = -(-remaining_rows // chunksize)  # Ceiling division
        estimated_time = (elapsed_time / rows_processed) * remaining_rows
        print(f"Estimated remaining time: {estimated_time:.2f} seconds or approximately {estimated_time/60:.2f} minutes.")


Processed 100000 rows in 31.42 seconds.
Estimated remaining time: 8768.42 seconds or approximately 146.14 minutes.
Processed 200000 rows in 61.90 seconds.
Estimated remaining time: 8606.12 seconds or approximately 143.44 minutes.
Processed 300000 rows in 97.19 seconds.
Estimated remaining time: 8976.12 seconds or approximately 149.60 minutes.
Processed 400000 rows in 132.00 seconds.
Estimated remaining time: 9110.11 seconds or approximately 151.84 minutes.
Processed 500000 rows in 163.65 seconds.
Estimated remaining time: 9002.87 seconds or approximately 150.05 minutes.
Processed 600000 rows in 194.34 seconds.
Estimated remaining time: 8876.56 seconds or approximately 147.94 minutes.
Processed 700000 rows in 224.44 seconds.
Estimated remaining time: 8755.01 seconds or approximately 145.92 minutes.
Processed 800000 rows in 253.77 seconds.
Estimated remaining time: 8629.88 seconds or approximately 143.83 minutes.
Processed 900000 rows in 283.33 seconds.
Estimated remaining time: 8533.05 

In [44]:
#read processed_df
processed_df = pd.read_csv('Master Thesis Data\\processed_df.csv')
processed_df

,Unnamed: 0,Post_Body,Post_Body_clean
0,0,I'm taking a course on Ruby on Rails. In the v...,im taking course ruby rail video instructor re...
1,1,I have an ERP system which generates statement...,erp system generates statement via word docx t...
2,2,Given this example from plotly documentation\n...,given example plotly documentation displayed c...
3,3,I have a folder structure like this:\n\nI have...,folder structure like referenced datajs appjs ...
4,4,In my Android Studio project I have a fragment...,android studio project fragment created displa...
...,...,...,...
2906359,3040725,"When reading the Opcodes for the EVM article, ...",reading opcodes evm article saw opcodes evalua...
2906360,3040726,am building Dapp based on microservice and am ...,building dapp based microservice testing somet...
2906361,3040727,"so have a contract deployed already, and ETH i...",contract deployed already eth autosent wallet ...
2906362,3040728,I have two contracts that implement . One is ...,two contract implement one marked marked creat...


In [48]:
# #drop Post_Body column
# processed_df = processed_df.drop(['Post_Body'], axis=1)

In [49]:
#merge processed_df with text_df (left join on Unnamed: 0)
df_clean = pd.merge(unprocessed_df, processed_df, on='Unnamed: 0', how='left')

In [51]:
df_clean

,Unnamed: 0,Post_Creation_Date,Post_View_Count,Post_Title,Post_Score,Post_Answer_Count,Response_Scores,Post_Author,Answer_Author,Forum,Week_of_Year,tag_1,tag_2,tag_3,Post_Body_clean
0,0,2021-07-01 00:00:58,5867,"railsinstaller.org website was not found, what...",1,3,"1,0,0",8530236,8530236,Stack Overflow,26,ruby-on-rails,ssl,NaN,im taking course ruby rail video instructor re...
1,1,2021-07-01 00:01:10,11,Insert page break if MS Word document is endin...,0,0,No Responses,3893947,0,Stack Overflow,26,ms-word,NaN,NaN,erp system generates statement via word docx t...
2,2,2021-07-01 00:01:18,1393,Plotly: How to show subplot titles at the bottom,1,1,2,2313889,13107804,Stack Overflow,26,python,plotly,subplot,given example plotly documentation displayed c...
3,3,2021-07-01 00:01:28,660,Javascript files not loading in HTML page,1,2,"0,1",15301811,4014834,Stack Overflow,26,javascript,html,d3.js,folder structure like referenced datajs appjs ...
4,4,2021-07-01 00:02:36,59,How can I put a fragment inside of a tab?,0,2,"1,1",14595783,1440565,Stack Overflow,26,java,android,xml,android studio project fragment created displa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040725,3040725,2023-04-30 16:46:47,3,How do the LT and GT opcodes differ from SLT a...,0,1,0,116681,0,Ethereum,17,solidity,evm,opcode,reading opcodes evm article saw opcodes evalua...
3040726,3040726,2023-04-30 17:21:18,15,is it possible to give metamask a raw transact...,0,0,No Responses,115531,0,Ethereum,17,metamask,web3.py,NaN,building dapp based microservice testing somet...
3040727,3040727,2023-04-30 21:02:45,41,Sending ETH to Gnosis Safe,0,0,No Responses,98043,0,Ethereum,17,gnosis-safe,NaN,NaN,contract deployed already eth autosent wallet ...
3040728,3040728,2023-04-30 22:03:50,11,How to override a function when two implementi...,0,1,0,37215,0,Ethereum,17,function-override,NaN,NaN,two contract implement one marked marked creat...


In [52]:
# #Everything seems correct! Now we drop the column Unnamed: 0 can 
# df_clean = df_clean.drop(['Unnamed: 0'], axis=1)
#save the df_clean
df_clean.to_csv('Master Thesis Data\\df_clean.csv', index=False)

In [53]:
end_time = time.time()
total_time = end_time - start_time
print("Total execution time: {:.2f} seconds".format(total_time))

Total execution time: 48232.87 seconds
